In [4]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [8]:
target_size = (224, 224)
seed = 42
path_dataset = r'/mnt/c/Datasets/Chest X-Ray Images/chest_xray'

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=10,
    brightness_range=[0.7,1.3],
    #validation_split=0.2
    )
train_generator = train_datagen.flow_from_directory(
    (path_dataset + '/train'),
    seed=seed,
    target_size=target_size,
    batch_size=32,
    class_mode='binary',
    #subset='training'
    )

val_datagen = ImageDataGenerator(
    rescale=1./255, 
    validation_split=0.2
)
validation_generator = val_datagen.flow_from_directory(
    (path_dataset + '/val'),
    seed=seed,
    target_size=target_size,
    batch_size=16,
    class_mode='binary',
    #subset='validation'
    )

Found 5216 images belonging to 2 classes.
Found 16 images belonging to 2 classes.
